In [1]:
import numpy as np


SAMPLE_SIZE = 10_000
np.random.seed(2025)  # Return same sample every time

# random_noise = np.random.normal(0, 0.1, SAMPLE_SIZE)

x1 = np.random.uniform(0, 1, SAMPLE_SIZE)
x2 = np.random.uniform(0, 1, SAMPLE_SIZE)
y = x1**2 + np.exp(np.sin(x2))
y = y.reshape((SAMPLE_SIZE, -1))

In [2]:
class NeuralNetwork:
    def __init__(self, layers, activation_functions):
        self.layers = layers
        self.activation_functions = activation_functions
        self._initialize_weights()
    
    @property
    def _initialize_weights(self):
        for i, j in zip(self.layers, self.layers[1:]):
            self.weights_and_biases = [np.ones(shape=(i + 1, j))]
        self.weights_and_biases = np.array(self.weights_and_biases)
    
    def inference(self, X: np.ndarray):
        assert X.shape[1] == self.layers[0]

        bias_column = np.ones(X.shape[0])
        X_inference = np.hstack((X, bias_column))

        current_features = X_inference
        for weights, activation_function in zip(self.weights_and_biases, self.activation_functions, ):
            h_star = np.matmul(current_features, weights)
            current_features = activation_function(h_star)
        
        return current_features

In [3]:
FEATURES = 2
HIDDEN_LAYERS_SIZES = [4]
OUPUTS = 1

xh_weights = np.ones(shape=(HIDDEN_LAYERS_SIZES[0], FEATURES)).T
xh_biases = np.zeros(shape=(HIDDEN_LAYERS_SIZES[0]))

hy_weights = np.ones(shape=(OUPUTS, HIDDEN_LAYERS_SIZES[-1])).T
hy_biases = np.zeros(shape=(OUPUTS))

In [4]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def feed_forward(x, xh_weights, xh_biases, hy_weights, hy_biases):
    h_star = np.matmul(x, xh_weights) + xh_biases
    h = sigmoid(h_star)

    y_star = np.matmul(h, hy_weights) + hy_biases
    y = sigmoid(y_star)

    return y

X = np.array([x1, x2]).T.reshape((SAMPLE_SIZE, FEATURES))
y_hat = feed_forward(X, xh_weights, xh_biases, hy_weights, hy_biases)

def MSE(y, y_hat):
    return np.mean(np.array(y - y_hat)**2)

MSE(y, y_hat)

np.float64(1.2619112958266134)

In [5]:
print(f"""
Mean: {np.mean(y):.3f}
Min: {np.min(y):.3f}
Max: {np.max(y):.3f}
""".strip())

Mean: 1.964
Min: 1.003
Max: 3.299


In [6]:
def feed_forward(x, xh_weights, xh_biases, hy_weights, hy_biases):
    h_star = np.matmul(x, xh_weights) + xh_biases
    h = sigmoid(h_star)

    y_star = np.matmul(h, hy_weights) + hy_biases
    y = y_star

    return y

y_hat = feed_forward(X, xh_weights, xh_biases, hy_weights, hy_biases)
MSE(y, y_hat)

np.float64(0.9100372271588312)

In [7]:
def backpropagation(X, y, xh_weights, xh_biases, hy_weights, hy_biases, lr):
    # Inference
    h_star = np.array(np.matmul(X, xh_weights) + xh_biases)
    h = sigmoid(h_star)

    y_star = np.array(np.matmul(h, hy_weights) + hy_biases)
    y_hat = y_star

    # Derivatives
    dy_yhat = 2 / SAMPLE_SIZE * (y - y_hat)
    dystar_hstar = h * (1 - h)
    dystar_hstar = np.array(dystar_hstar * hy_weights.reshape(1, -1))
    dystar_hstar = dy_yhat * dystar_hstar

    # Update weights and biases
    hy_biases += lr * np.sum(dy_yhat, axis=0)
    hy_weights += lr * np.sum(dy_yhat * h, axis=0).reshape(-1, 1)

    xh_biases += lr * np.sum(dystar_hstar, axis=0)
    xh_weights += lr * np.matmul(X.T, dystar_hstar)

    return MSE(y, y_hat)

In [8]:
EPOCHS = 100
losses = []
for i in range(EPOCHS):
    ith_loss = backpropagation(X, y, xh_weights, xh_biases, hy_weights, hy_biases, 0.01)
    losses.append(ith_loss)

In [12]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(list(range(EPOCHS)), losses)
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
ax.set(xlabel="Epoch", ylabel="MSE")

plt.savefig(f"assets/images/training.png", transparent=True)
plt.close()